In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('test').getOrCreate()

In [5]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import format_number

def crosstabPercentage(df, col1, col2):
    ctabDf = df.crosstab(col1, col2)
    ctabCol = col1 + '_' + col2
    ctabNewCol = col1 + col2.title()
    ctabDf = ctabDf.withColumn(ctabNewCol, ctabDf[ctabCol])\
                                         .orderBy(ctabNewCol).drop(ctabCol)
    # Strip extra whitespaces from column name
    for column in ctabDf.columns:
        columnStripped = column.strip()
        if column != columnStripped:
            ctabDf = ctabDf.withColumn(column.strip(), ctabDf[column])\
                                         .drop(column)

#     ctabDf = ctabDf.withColumn('percentage-of->50K', ctabDf['>50K']/(ctabDf['<=50K']+ctabDf['>50K'])*100)
#     return ctabDf
    ctabDf.show()

In [23]:
from pyspark.sql.functions import col, lit
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (1, 2, 3),
    (2, 2, 3),
    (4, 1, 3),
    (4, 2, 3),
], ["label", "id2", "id3"])
labelCol = "label"

from pyspark.sql.window import Window

w =  Window.partitionBy(df[labelCol]).orderBy(df['id2'])
w
# df.crosstab("id1", "id2").show()
# crosstabPercentage(df, "id1", "id2")

# df2 = df.groupby(labelCol, "id2").count()
# count = df.count()
# df2.withColumn('percentage', col('count') / count).show()

In [17]:
grouped = df.groupby("id")
grouped

In [ ]:
grouped.

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest
from pyspark.sql.functions import min, max

data = [(0.0, Vectors.dense(0.5, 10.0, 10.0)),
        (0.0, Vectors.dense(1.5, 20.0, 10.0)),
        (1.0, Vectors.dense(1.5, 30.0, 20.0)),
        (0.0, Vectors.dense(3.5, 30.0, 10.0)),
        (0.0, Vectors.dense(3.5, 40.0, 10.0)),
        (1.0, Vectors.dense(3.5, 40.0, 20.0))]
df = spark.createDataFrame(data, ["label", "features"])

r = ChiSquareTest.test(df, "features", "label").head()
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))


pValues: [0.687289278791,0.682270330336,0.0143058784354]
degreesOfFreedom: [2, 3, 1]
statistics: [0.75,1.5,6.0]


In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = [(2, Vectors.dense(0.5, 10.0)),
        (0, Vectors.dense(1.5, 20.0)),
        (1, Vectors.dense(1.5, 30.0)),
        (2, Vectors.dense(3.5, 30.0)),
        (1, Vectors.dense(3.5, 40.0)),
        (1, Vectors.dense(3.5, 40.0))]
data = spark.createDataFrame(data, ["label", "features"])

# Split the data into train and test
splits = data.randomSplit([0.6, 0.4], 1234)
# train = splits[0]
# test = splits[1]

train = data

# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [2, 5, 4, 3]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(train)

# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))